In [10]:
from keras.preprocessing.image import ImageDataGenerator
import keras

In [11]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [12]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model

In [13]:
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)
# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

In [14]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

# add your top layer block to your base model
model = Model(base_model.input, predictions)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [15]:
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 16666 images belonging to 120 classes.
Found 3911 images belonging to 120 classes.


In [16]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

In [17]:
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator)

Epoch 1/10
521/521 [==============================] - 2001s 4s/step - loss: 4.1501 - accuracy: 0.4283 - val_loss: 3.0777 - val_accuracy: 0.7750
Epoch 2/10
521/521 [==============================] - 1960s 4s/step - loss: 1.8155 - accuracy: 0.8178 - val_loss: 0.8782 - val_accuracy: 0.8645
Epoch 3/10
521/521 [==============================] - 1938s 4s/step - loss: 0.6989 - accuracy: 0.8646 - val_loss: 0.5248 - val_accuracy: 0.8814
Epoch 4/10
521/521 [==============================] - 1939s 4s/step - loss: 0.5150 - accuracy: 0.8727 - val_loss: 0.4434 - val_accuracy: 0.8821
Epoch 5/10
521/521 [==============================] - 1939s 4s/step - loss: 0.4422 - accuracy: 0.8781 - val_loss: 0.4059 - val_accuracy: 0.8844
Epoch 6/10
521/521 [==============================] - 1940s 4s/step - loss: 0.4095 - accuracy: 0.8826 - val_loss: 0.3849 - val_accuracy: 0.8826
Epoch 7/10
521/521 [==============================] - 1949s 4s/step - loss: 0.3816 - accuracy: 0.8881 - val_loss: 0.3661 - val_accuracy:

In [18]:
model.save('sanford_dog_xception.h5')

In [19]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')

123/123 - 415s - loss: 0.3468 - accuracy: 0.8901 - 415s/epoch - 3s/step

Test accuracy: 0.8900536894798279
